# Otimizacao de Budget e Planejamento de Cenarios

Notebook focado na otimizacao de alocacao de marketing e simulacao de cenarios.


In [ ]:
import sys
sys.path.insert(0, "..")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from loguru import logger


## 1. Preparacao

Carregamos dados, aplicamos transformacoes e treinamos o modelo.


In [ ]:
from src.ingestion.loader import DataLoader
from src.features.adstock import AdstockTransformer
from src.features.saturation import SaturationTransformer
from src.features.engineering import FeatureEngineer
from src.models.regularized import RegularizedMMM
from src.config import MEDIA_CHANNELS

loader = DataLoader()
df = loader.generate_sample_data(n_weeks=104)

adstock = AdstockTransformer(method="geometric")
df = adstock.transform(df)

saturation = SaturationTransformer(method="exponential")
df = saturation.transform(df)

engineer = FeatureEngineer()
df = engineer.create_date_features(df)
df = df.dropna().reset_index(drop=True)

feature_cols = [
    c for c in df.select_dtypes(include=[np.number]).columns
    if c not in ("revenue", "date")
]

mmm = RegularizedMMM(model_type="ridge", alpha=1.0)
X_train, X_test, y_train, y_test = mmm.prepare_data(df, feature_cols)
mmm.fit(X_train, y_train)
metrics = mmm.evaluate(X_test, y_test)
print(f"R2: {metrics["r2"]:.4f}")

## 2. Otimizacao de Budget


In [ ]:
from src.optimization.budget import BudgetOptimizer

base_features = df[feature_cols].mean().values
media_in_features = [c for c in MEDIA_CHANNELS if c in feature_cols]

optimizer = BudgetOptimizer(
    model=mmm.model,
    scaler=mmm.scaler,
    feature_names=feature_cols,
    media_channels=media_in_features,
)

budgets = np.arange(200000, 1000001, 100000)
range_results = optimizer.optimize_range(budgets, base_features)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=range_results["budget"],
    y=range_results["predicted_revenue"],
    mode="lines+markers",
))
fig.update_layout(
    title="Receita Prevista por Nivel de Budget",
    xaxis_title="Budget Total (R$)",
    yaxis_title="Receita Prevista (R$)",
    template="plotly_white",
)
fig.show()

## 3. ROAS Marginal por Canal


In [ ]:
fig = go.Figure()

for channel in media_in_features[:4]:
    mroas = optimizer.marginal_roas(base_features, channel)
    fig.add_trace(go.Scatter(
        x=mroas["increment"],
        y=mroas["marginal_roas"],
        name=channel.replace("_spend", ""),
        mode="lines",
    ))

fig.update_layout(
    title="ROAS Marginal por Incremento de Investimento",
    xaxis_title="Incremento (R$)",
    yaxis_title="ROAS Marginal",
    template="plotly_white",
)
fig.show()

## 4. Cenarios de Alocacao


In [ ]:
from src.optimization.scenarios import ScenarioPlanner

planner = ScenarioPlanner(
    model=mmm.model,
    scaler=mmm.scaler,
    feature_names=feature_cols,
    media_channels=media_in_features,
)

planner.create_baseline(base_features)
planner.create_proportional_scenario(base_features, 0.7, "Corte 30%")
planner.create_proportional_scenario(base_features, 1.0, "Atual")
planner.create_proportional_scenario(base_features, 1.3, "Aumento 30%")

if len(media_in_features) >= 2:
    planner.create_channel_focus_scenario(
        base_features,
        focus_channel=media_in_features[0],
        focus_multiplier=2.0,
        name="Foco Canal 1",
    )
    planner.create_reallocation_scenario(
        base_features,
        from_channel=media_in_features[-1],
        to_channel=media_in_features[0],
        transfer_pct=0.5,
        name="Realocacao",
    )

comparison = planner.compare_scenarios()
comparison

## 5. Analise de Sensibilidade


In [ ]:
from src.visualization.charts import MMMCharts

charts = MMMCharts()

if media_in_features:
    sensitivity = planner.sensitivity_analysis(
        base_features,
        channel=media_in_features[0],
        multiplier_range=np.arange(0.0, 3.1, 0.1),
    )
    fig = charts.plot_sensitivity(sensitivity, media_in_features[0])
    fig.show()

## 6. Exportacao dos Resultados


In [ ]:
from src.visualization.export import ResultExporter

exporter = ResultExporter()

print("Resultados prontos para exportacao.")
print("Em producao, use:")
print("  exporter.export_model_summary(metrics, coefs, "ridge")")
print("  exporter.export_scenarios(comparison)")

---

*"O futuro pertence a quem se prepara para ele." - Ralph Waldo Emerson*
